### 本节目标 
- 理解和实现softmax分类模型 
- 搭建softmax分类模型神经网络

softmax分类的本质依然是通过线性回归的方式实现分类，通过将线性回归的输出值变为各个分类概率实现分类。
具体介绍见 http://tangshusen.me/Dive-into-DL-PyTorch/#/chapter03_DL-basics/3.4_softmax-regression

### 读取数据集

数据集采用 Fashion-MNIST 图像分类数据集

In [1]:
import torch
import torchvision
# torchvision包  用来构建计算机视觉模型。orchvision主要由以下几部分构成：
# torchvision.datasets: 一些加载数据的函数及常用的数据集接口；
# torchvision.models: 包含常用的模型结构（含预训练模型），例如AlexNet、VGG、ResNet等；
# torchvision.transforms: 常用的图片变换，例如裁剪、旋转等；
# torchvision.utils: 其他的一些有用的方法。

import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import time
import sys  # sys库详解  https://www.cnblogs.com/Archie-s/p/6860301.html
sys.path.append("..") # 为了导入上层目录的d2lzh_pytorch
# import d2lzh_pytorch as d2l

In [2]:
# 通过torchvision的torchvision.datasets来下载这个数据集
# train来指定获取训练数据集或测试数据集
# transform = transforms.ToTensor()使所有数据转换为Tensor，如果不进行转换则返回的是PIL图片
# root 指定下载路径
mnist_train = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=True, download=True, transform=transforms.ToTensor())
mnist_test = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=False, download=True, transform=transforms.ToTensor())

数据由10个类别的衣物图片组成，每个类别的训练集和测试集分别是6000和1000

In [3]:
print(type(mnist_train))
print(len(mnist_train), len(mnist_test))
feature, label = mnist_train[0]
print(feature.shape, label)

<class 'torchvision.datasets.mnist.FashionMNIST'>
60000 10000
torch.Size([1, 28, 28]) 9


In [4]:
# 读取数据
batch_size = 256
num_workers = 4
train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [5]:
start = time.time()
for X, y in train_iter:
    continue
print('%.2f sec' % (time.time() - start))

4.30 sec


读取数据德操作与上一节一样使用DataLoader

### softmax从零实现

In [6]:
import torch
import torchvision
import numpy as np
import sys
sys.path.append("..") # 为了导入上层目录的d2lzh_pytorch
# import d2lzh_pytorch as d2l

读取数据

In [7]:
batch_size = 256
num_workers = 4
train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=num_workers)

初始化模型参数

每个样本是28*28的像素，有10个类别，所以输入是784 输出是10  模型是 y=softmax(X * w + b)=softmax((10 * 784) * (784 * 1) + b)

In [8]:
num_inputs = 784
num_outputs = 10

W = torch.tensor(np.random.normal(0, 0.01, (num_inputs, num_outputs)), dtype=torch.float)
b = torch.zeros(num_outputs, dtype=torch.float)

In [9]:
W.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True) 

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

实现softmax运算

正常的线性回归模型，每一个输出都是一个连续值，为明确最终的分类结果，softmax将每一个分类的连续值变成占所有分类的百分比概率输出

详述 http://tangshusen.me/Dive-into-DL-PyTorch/#/chapter03_DL-basics/3.4_softmax-regression

在进行softmax运算前，特征已经经过线性回归处理，输出了每个输出的值，所以矩阵X的行数是样本数，列数是输出个数。

In [10]:
# softmax运算会先通过exp函数对每个元素做指数运算，
# 再对exp矩阵同行元素求和
# 最后令矩阵每行各元素与该行元素之和相除
# 最终得到的矩阵每行元素和为1且非负。因此，该矩阵每行都是合法的概率分布。
# softmax运算的输出矩阵中的任意一行元素代表了一个样本在各个输出类别上的预测概率。
def softmax(X):
    X_exp = X.exp() 
    partition = X_exp.sum(dim=1, keepdim=True) # 求和
    return X_exp / partition  # 这里应用了广播机制

定义模型

In [11]:
def net(X):
    return softmax(torch.mm(X.view((-1, num_inputs)), W) + b)
#   (1*785) * (785*10) + 1*1

定义损失函数 

softmax回归使用的交叉熵损失函数

如果预测值（0.3，0.1，0.6） 真实值（0，0，1） 交叉熵 = -1*log0.6

In [12]:
def cross_entropy(y_hat, y):
    return - torch.log(y_hat.gather(1, y.view(-1, 1)))

计算分类准确率

In [15]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用。该函数将被逐步改进：它的完整实现将在“图像增广”一节中描述
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        # y_hat.argmax(dim=1)返回矩阵y_hat每行中最大元素的索引
        # 与y比较 索引相同即判断正确为1，不等为0
        n += y.shape[0]
    return acc_sum / n
        # 求平均便是准确率

训练模型

训练softmax回归的实现跟线性回归中的实现非常相似。我们同样使用小批量随机梯度下降来优化模型的损失函数。
因为梯度是在反向传播中求出的，所以随机梯度下降依然适用

In [16]:
def sgd(params, lr, batch_size): 
    for param in params:
        param.data -= lr * param.grad / batch_size # .data 返回和 x 的相同数据 tensor, 但不会加入到x的计算历史里

In [18]:
num_epochs, lr = 5, 0.1

# 本函数已保存在d2lzh包中方便以后使用
def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size,
              params=None, lr=None, optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        # train_l_sum训练集整体交叉熵损失值 train_acc_sum训练集准确率
        for X, y in train_iter:
            y_hat = net(X)             # softmax模型运算
            l = loss(y_hat, y).sum()   # 损失函数

            # 梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()

            l.backward()               # 梯度计算
            if optimizer is None:
                sgd(params, lr, batch_size)  #优化
            else:
                optimizer.step()  # “softmax回归的简洁实现”一节将用到


            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, batch_size, [W, b], lr)

epoch 1, loss 0.7871, train acc 0.749, test acc 0.796
epoch 2, loss 0.5692, train acc 0.814, test acc 0.814
epoch 3, loss 0.5254, train acc 0.826, test acc 0.819
epoch 4, loss 0.5017, train acc 0.833, test acc 0.824
epoch 5, loss 0.4857, train acc 0.837, test acc 0.827


### softmax pytorch 简洁实现

In [19]:
import torch
from torch import nn
from torch.nn import init
import numpy as np

In [21]:
batch_size = 256
train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=num_workers)

softmax回归的输出层是一个全连接层，所以我们用一个线性模块就可以了。因为前面我们数据返回的每个batch样本x的形状为(batch_size, 1, 28, 28), 所以我们要先用view()将x的形状转换成(batch_size, 784)才送入全连接层。

In [22]:
num_inputs = 784
num_outputs = 10

class LinearNet(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(num_inputs, num_outputs)
    def forward(self, x): # x shape: (batch, 1, 28, 28) 前向传播，矩阵相乘计算的过程
        y = self.linear(x.view(x.shape[0], -1))
        return y

net = LinearNet(num_inputs, num_outputs)

还可以用容器定义模型

首先将对x的形状转换的这个功能自定义一个FlattenLayer

In [23]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
class FlattenLayer(nn.Module):
    def __init__(self):
        super(FlattenLayer, self).__init__()
    def forward(self, x): # x shape: (batch, *, *, ...)
        return x.view(x.shape[0], -1)

In [24]:
from collections import OrderedDict

net = nn.Sequential(
    # FlattenLayer(),
    # nn.Linear(num_inputs, num_outputs)
    OrderedDict([
        ('flatten', FlattenLayer()),                   #先变幻形状
        ('linear', nn.Linear(num_inputs, num_outputs)) #做线性回归
    ])
)

初始化模型参数

In [25]:
init.normal_(net.linear.weight, mean=0, std=0.01)
init.constant_(net.linear.bias, val=0) 

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

分开定义softmax运算和交叉熵损失函数可能会造成数值不稳定。因此，PyTorch提供了一个包括softmax运算和交叉熵损失计算的函数。它的数值稳定性更好。

In [26]:
loss = nn.CrossEntropyLoss()

优化算法     我们使用学习率为0.1的小批量随机梯度下降作为优化算法。

In [27]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

训练模型

In [28]:
num_epochs = 5

# 本函数已保存在d2lzh包中方便以后使用
def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size,
              params=None, lr=None, optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        # train_l_sum训练集整体交叉熵损失值 train_acc_sum训练集准确率
        for X, y in train_iter:
            y_hat = net(X)             # 模型运算
            l = loss(y_hat, y).sum()   # 损失函数

            # 梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()

            l.backward()               # 梯度计算
            if optimizer is None:
                sgd(params, lr, batch_size)  #优化
            else:
                optimizer.step()  # “softmax回归的简洁实现”一节将用到


            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net) ####################
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))


In [29]:
train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None, None, optimizer)

epoch 1, loss 0.0031, train acc 0.751, test acc 0.790
epoch 2, loss 0.0022, train acc 0.813, test acc 0.810
epoch 3, loss 0.0021, train acc 0.825, test acc 0.818
epoch 4, loss 0.0020, train acc 0.833, test acc 0.825
epoch 5, loss 0.0019, train acc 0.837, test acc 0.827


在最后使用测试集预测，判断预测准确率时，没有用到softmax运算。

softmax运算的作用是在训练模型计算loss时统一量纲。在最终预测时并不需要。